In [2]:
import numpy as np
import pandas as pd
import pandas_market_calendars as mcal
from sklearn.ensemble import RandomForestRegressor, AdaBoostRegressor, GradientBoostingRegressor
from sklearn.linear_model import ElasticNet
from sklearn.metrics import accuracy_score, confusion_matrix, f1_score, precision_score, recall_score
from sklearn.metrics import mean_squared_error, median_absolute_error
from sklearn.model_selection import GridSearchCV, PredefinedSplit
from sklearn.neural_network import MLPRegressor
from sklearn.preprocessing import StandardScaler, QuantileTransformer
from sklearn.svm import SVR

In [3]:
tickers = list([
    "SPY",  # S&P 500 Index Fund
    "IWV",  # Russell 3000 Index Fund
    "QQQ",  # Technology Sector Fund
    "IYF",  # Financials Sector Fund
    "XLP",  # Consumer Staples Sector Fund
    "XLU",  # Utilities Sector Funds
    "XLV",  # Health Care Sector Funds
    "IGE",  # NA Natural Resources ETF
    "XLE"  # Energy Sector Fund
])

indicators = {
    "3M_TBILL": "DTB3",  # 3-Month Treasury Bill: Secondary Market Rate
    "CPI": "MEDCPIM158SFRBCLE",  # Median Consumer Price Index
    "VIX": "VIXCLS",  # CBOE Volatility Index
    "INDP": "INDPRO",  # Industrial Production: Total Index
    "USHY_ADJ": "BAMLH0A0HYM2",  # ICE BofA US High Yield Index Option-Adjusted Spread
    "US_LEADING": "USSLIND",  # Leading Index for the United States
    "30Y_FRMTG": "MORTGAGE30US",  # 30-Year Fixed Rate Mortgage Average in the United States
    "15Y_FRMTG": "MORTGAGE15US",  # 15-Year Fixed Rate Mortgage Average in the United States
    "CPI_URBAN": "CUSR0000SEHA",  # Consumer Price Index for All Urban Consumers: Rent of Primary Residence in U.S. City Average
    "RETAIL": "RSAFS",  # Advance Retail Sales: Retail and Food Services, Total
    "PHARMA": "PCU32543254",  # Producer Price Index by Industry: Pharmaceutical and Medicine Manufacturing
    "UNEMP": "UNRATE",  # Unemployment Rate
    "UNEMP_PERM": "LNS13026638",  # Unemployment Level - Permanent Job Losers
    "UNEMP_MEN": "LNS14000001",  # Unemployment Rate - Men
    "UNEMP_WMN": "LNS14000002",  # Unemployment Rate - Women
    "UNEMP_WHT": "LNS14000003",  # Unemployment Rate - White
    "UNEMP_BLK": "LNS14000006",  # Unemployment Rate - Black or African American
    "UNEMP_HIS": "LNS14000009",  # Unemployment Rate - Hispanic or Latino
    "INC": "PI",  # Personal Income
    "INC_DISP": "DSPIC96",  # Real Disposable Personal Income
    "INC_DISP_PC": "A229RX0",  # Real Disposable Personal Income: Per Capita
    "TAX_HIGH": "IITTRHB",  # U.S Individual Income Tax: Tax Rates for Regular Tax: Highest Bracket
    "TAX_LOW": "IITTRLB"  # U.S Individual Income Tax: Tax Rates for Regular Tax: Lowest Bracket
}

features = []
for ticker in tickers:
    features.append(f'{ticker}_1D_RET')
    features.append(f'{ticker}_1D_VOL')
    for timeframe in ['1W', '1M', '3M', '6M', '1Y']:
        for calculation in ['RET', 'STD', 'VOL']:
            features.append(f'{ticker}_{timeframe}_{calculation}')
for indicator in indicators.keys():
    features.append(indicator)

"""
We will use all 25 target columns in the neural network, for now we will use only the SPY 1 month return
targets = [f'{ticker}_TARGET' for ticker in tickers] +\
    ['3M_TBILL', 'CPI', 'VIX', 'INDP', 'USHY_ADJ', '30Y_FRMTG', '15Y_FRMTG', 'RETAIL', 'PHARMA', 'UNEMP', 'INC']
"""
targets = ['SPY_TARGET']


# construct a dictionary with all market data in divided into sets and features/targets
dates = mcal.get_calendar('NYSE').schedule(start_date='2004-01-01', end_date='2020-12-31').index
market_data = dict({
    "X" : pd.read_pickle("data/market_data.zip").loc[:, features],
    "y" : pd.read_pickle("data/market_data.zip").loc[:, targets]
})
market_data["X_train"] = market_data["X"].loc['2004-01-01':'2015-12-31', :]
market_data["y_train"] = market_data["y"].loc['2004-01-01':'2015-12-31', :]
market_data["X_test"] = market_data["X"].loc['2016-01-01':'2020-12-31', :]
market_data["y_test"] = market_data["y"].loc['2016-01-01':'2020-12-31', :]
market_data["X"] = market_data["X"].loc['2004-01-01':'2020-12-31', :]
market_data["y"] = market_data["y"].loc['2004-01-01':'2020-12-31', :]

# Create split on train_all with -1 for training data and 0 for validation data (data after '2013-01-01')
split = PredefinedSplit(test_fold=[0 if v else -1 for v in market_data["X_train"].index < '2013-01-01'])

## Transform and Scale the Data

In [4]:
market_data["X_train"].shape

(4383, 176)

In [5]:
# add in quantiles as additional feature columns
quantile_transformer = QuantileTransformer()
quantile_transformer.fit(market_data["X_train"])
market_data["X_train"].loc[:, [col + "_QUANTILE" for col in features]] = pd.DataFrame(
    quantile_transformer.transform(market_data["X_train"]), index=market_data["X_train"].index,
    columns=[col + "_QUANTILE" for col in features])
market_data["X_test"].loc[:, [col + "_QUANTILE" for col in features]] = pd.DataFrame(
    quantile_transformer.transform(market_data["X_test"]), index=market_data["X_test"].index,
    columns=[col + "_QUANTILE" for col in features])

# scale all data based on the training set
X_scaler = StandardScaler()
y_scaler = StandardScaler(with_mean=False)
X_scaler.fit(market_data["X_train"])
y_scaler.fit(market_data["y_train"])

X_train = pd.DataFrame(X_scaler.transform(market_data["X_train"]),
                       index=market_data["X_train"].index, columns=market_data["X_train"].columns)
y_train = pd.DataFrame(y_scaler.transform(market_data["y_train"]),
                       index=market_data["y_train"].index, columns=market_data["y_train"].columns)
X_test = pd.DataFrame(X_scaler.transform(market_data["X_test"]),
                      index=market_data["X_test"].index, columns=market_data["X_test"].columns)
y_test = pd.DataFrame(y_scaler.transform(market_data["y_test"]),
                      index=market_data["y_test"].index, columns=market_data["y_test"].columns)
assert not any([any(arr) for arr in np.array(np.isinf(X_train))])
assert not any([any(arr) for arr in np.array(np.isnan(X_train))])
assert not any([any(arr) for arr in np.array(np.isinf(X_test))])
assert not any([any(arr) for arr in np.array(np.isnan(X_test))])
assert not any([any(arr) for arr in np.array(np.isinf(y_train))])
assert not any([any(arr) for arr in np.array(np.isnan(y_train))])
assert not any([any(arr) for arr in np.array(np.isinf(y_test))])
assert not any([any(arr) for arr in np.array(np.isnan(y_test))])

In [6]:
def to_signal(x):
    return np.array([1 if 0 <= yt else -1 for yt in np.array(x).ravel()])

def classification_metrics(y_true, y_pred):
    """
    A method to evaluate the true return values versus the predicted value as a binary classification task, where
    values greater than 0 are set to 1, values under 0 are set to -1

    :param y_true: true 1-month look-ahead return values
    :param y_pred: predicted 1-month look-ahead return values
    """
    true_classes = to_signal(y_true)#.reshape(-1, 1)
    pred_classes = to_signal(y_pred)#.reshape(-1, 1)

    accuracy = accuracy_score(true_classes, pred_classes)
    cnf_mat = confusion_matrix(true_classes, pred_classes, labels=[1, -1])
    print(f"Confusion Matrix\n{cnf_mat}")
    print(f"Accuracy            = {accuracy:.04f}\n"
          f"F1-Score (Micro)    = {f1_score(true_classes, pred_classes, average='micro'):.04f}\n"
          f"F1-Score (Macro)    = {f1_score(true_classes, pred_classes, average='macro'):.04f}\n"
          f"F1-Score (Weighted) = {f1_score(true_classes, pred_classes, average='weighted'):.04f}\n")

In [13]:
models = {
    'Linear Regression': GridSearchCV(estimator=ElasticNet(max_iter=100000, tol=0.004), param_grid={
        'l1_ratio': [0.4, 0.5, 0.6]
    }, cv=split, scoring="explained_variance", n_jobs=-1),
    'Support Vector Machine': GridSearchCV(estimator=SVR(), param_grid={
        'kernel': ["linear", "rbf", "poly"],
        'degree': [2, 3, 4]
    }, cv=split, scoring="explained_variance", n_jobs=-1),
    'Random Forest': GridSearchCV(estimator=RandomForestRegressor(bootstrap=True, n_jobs=-1), param_grid={
        "n_estimators": [n for n in range(50, 175, 25)],
        "ccp_alpha": [0.01, 0.05, 0.10]
    }, cv=split, scoring="explained_variance", n_jobs=-1),
    'Adaptive Boost': GridSearchCV(estimator=AdaBoostRegressor(), param_grid={
        "n_estimators": [n for n in range(50, 250, 50)],
        "loss": ["linear", "exponential"],
        "learning_rate": [0.01, 0.1, 1]
    }, cv=split, scoring="explained_variance", n_jobs=-1),
    'Gradient Boost': GridSearchCV(estimator=GradientBoostingRegressor(loss="huber"), param_grid={
        "n_estimators": [n for n in range(50, 250, 50)],
        "ccp_alpha": [0.01, 0.05, 0.10]
    }, cv=split, scoring="explained_variance", n_jobs=-1),
    'Neural Net': GridSearchCV(estimator=MLPRegressor(solver="lbfgs", max_iter=1000000), param_grid={
        "hidden_layer_sizes": [(300, h1, h2)
                               for h1 in range(100, 200, 50)
                               for h2 in range(25, h1//2, 25)]
    }, cv=split, scoring="explained_variance", n_jobs=-1)
}

In [14]:
for model_type, model in models.items():
    model.fit(X_train, np.array(y_train).ravel())

In [15]:
print(f"Results:"
      f"--------------------------------------------------")
pred_average = None   # regression average
pred_ensemble = None  # classification mean vote
for model_type, model in models.items():
    # regression results
    print(f"- Model: {model_type}\n"
          f"  - best parameters: {model.best_params_}")
    pred = model.predict(X_test)
    if pred_average is None: pred_average = np.array(pred) / len(models)
    else: pred_average = pred_average + (np.array(pred) / len(models))
    print(f"  - Regression:\n"
          f"    - MSE: {mean_squared_error(np.array(y_test), np.array(pred).ravel()):.06f}\n"
          f"    - MAE: {median_absolute_error(np.array(y_test), np.array(pred).ravel()):.06f}")

    # classification results
    pred = [-1 if i < 0 else 1 for i in np.array(pred).ravel()]
    if pred_ensemble is None: pred_ensemble = np.array(pred) / len(models)
    else: pred_ensemble = pred_ensemble + (np.array(pred) / len(models))
    conf_mat = confusion_matrix(
        [0 if i < 0 else 1 for i in y_test.to_numpy().ravel()],
        [0 if i < 0 else 1 for i in np.array(pred).ravel()])
    accuracy = accuracy_score(
        [0 if i < 0 else 1 for i in y_test.to_numpy().ravel()],
        [0 if i < 0 else 1 for i in np.array(pred).ravel()])
    precision = precision_score(
        [0 if i < 0 else 1 for i in y_test.to_numpy().ravel()],
        [0 if i < 0 else 1 for i in np.array(pred).ravel()])
    recall = recall_score(
        [0 if i < 0 else 1 for i in y_test.to_numpy().ravel()],
        [0 if i < 0 else 1 for i in np.array(pred).ravel()])
    f1 = f1_score(
        [0 if i < 0 else 1 for i in y_test.to_numpy().ravel()],
        [0 if i < 0 else 1 for i in np.array(pred).ravel()])
    print(f"  - Classification\n"
          f"    - Confusion Matrix:\n"
          f"      [{conf_mat[0]}\n"
          f"       {conf_mat[1]}]\n"
          f"    - Accuracy: {accuracy:.06f}\n"
          f"    - Recall: {recall:.06f}\n"
          f"    - Precision: {precision:.06f}\n"
          f"    - F1-Score: {f1:.06f}\n"
          f"--------------------------------------------------")

# ensemble of all benchmark models
# regression results
print(f"- Ensemble of all benchmark models")
print(f"  - Regression:\n"
      f"    - MSE: {mean_squared_error(np.array(y_test), np.array(pred_average).ravel()):.06f}\n"
      f"    - MAE: {median_absolute_error(np.array(y_test), np.array(pred_average).ravel()):.06f}\n"
      f"--------------------------------------------------")

# classification results
conf_mat = confusion_matrix(
    [0 if i < 0 else 1 for i in y_test.to_numpy().ravel()],
    [0 if i < 0 else 1 for i in np.array(pred_ensemble).ravel()])
accuracy = accuracy_score(
    [0 if i < 0 else 1 for i in y_test.to_numpy().ravel()],
    [0 if i < 0 else 1 for i in np.array(pred_ensemble).ravel()])
precision = precision_score(
    [0 if i < 0 else 1 for i in y_test.to_numpy().ravel()],
    [0 if i < 0 else 1 for i in np.array(pred_ensemble).ravel()])
recall = recall_score(
    [0 if i < 0 else 1 for i in y_test.to_numpy().ravel()],
    [0 if i < 0 else 1 for i in np.array(pred_ensemble).ravel()])
f1 = f1_score(
    [0 if i < 0 else 1 for i in y_test.to_numpy().ravel()],
    [0 if i < 0 else 1 for i in np.array(pred_ensemble).ravel()])
print(f"  - Classification\n"
      f"    - Confusion Matrix:\n"
      f"      [{conf_mat[0]}\n"
      f"       {conf_mat[1]}]\n"
      f"    - Accuracy: {accuracy:.06f}\n"
      f"    - Recall: {recall:.06f}\n"
      f"    - Precision: {precision:.06f}\n"
      f"    - F1-Score: {f1:.06f}\n"
      f"--------------------------------------------------")



Results:--------------------------------------------------
- Model: Linear Regression
  - best parameters: {'l1_ratio': 0.4}
  - Regression:
    - MSE: 1.264388
    - MAE: 0.484234
  - Classification
    - Confusion Matrix:
      [[  0 498]
       [   0 1329]]
    - Accuracy: 0.727422
    - Recall: 1.000000
    - Precision: 0.727422
    - F1-Score: 0.842205
--------------------------------------------------
- Model: Support Vector Machine
  - best parameters: {'degree': 2, 'kernel': 'rbf'}
  - Regression:
    - MSE: 1.498640
    - MAE: 0.623448
  - Classification
    - Confusion Matrix:
      [[187 311]
       [602 727]]
    - Accuracy: 0.500274
    - Recall: 0.547028
    - Precision: 0.700385
    - F1-Score: 0.614280
--------------------------------------------------
- Model: Random Forest
  - best parameters: {'ccp_alpha': 0.1, 'n_estimators': 100}
  - Regression:
    - MSE: 2.130246
    - MAE: 0.500890
  - Classification
    - Confusion Matrix:
      [[ 53 445]
       [ 218 1111]]
 